<a href="https://colab.research.google.com/github/freddysundowner/shambaKit/blob/main/Final_FredModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q opendatasets

In [ ]:
import opendatasets as od
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import keras
from keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img
from keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions

In [ ]:
od.download('https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset') # insert ypu kaggle  username and key

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: alutatechnologies
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset


100%|██████████| 2.70G/2.70G [00:13<00:00, 220MB/s]


In [ ]:
list_crasses = os.listdir("/content/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train")
len(list_crasses)

38

In [ ]:
train_datagen = ImageDataGenerator(zoom_range=0.5, shear_range=0.3,horizontal_flip= True, preprocessing_function=preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train = train_datagen.flow_from_directory(
    directory= "/content/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train",
    target_size = (256,256),
    batch_size = 32)
val = val_datagen.flow_from_directory(
    directory= "/content/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid",
    target_size = (256,256),
    batch_size = 32)

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


In [ ]:
t_img, label = train.next()

In [ ]:
t_img.shape

(32, 256, 256, 3)

In [ ]:
def plotImage(img_arr, label):
  for im, l in zip(img_arr, label):
    plt.figure(figsize=(5,5))
    plt.show()

In [ ]:
plotImage(t_img[:3], label[:3])

<Figure size 500x500 with 0 Axes>

<Figure size 500x500 with 0 Axes>

<Figure size 500x500 with 0 Axes>

In [ ]:
from keras.layers import Dense, Flatten
from keras.models import Model
import keras

In [ ]:
base_model = VGG19(input_shape=(256,256,3), include_top= False)

80134624/80134624 [==============================] - 0s 0us/step


In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
base_model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [ ]:
x = Flatten()(base_model.output)
x = Dense(units = 38, activation = 'softmax')(x)
model = Model(base_model.input, x)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
es = EarlyStopping(
    monitor = 'val_accuracy',
    min_delta = 0.01,
    patience = 3,
    verbose = 1
)

mc = ModelCheckpoint(
        filepath = "best_model.h5",
        monitor = "val_accuracy",
        min_delta = 0.01,
        patience = 3,
        verbose = 1,
        save_best_only = True
)
cb = [es,mc]

In [ ]:
model.compile(
    optimizer="adam",
    loss = keras.losses.categorical_crossentropy,
    metrics = ['accuracy']
    )

In [ ]:
his = model.fit_generator(train,
                          steps_per_epoch =16,
                          epochs = 50,
                          verbose = 1,
                          callbacks = cb,
                          validation_data = val,
                          validation_steps = 16
)

<ipython-input-20-7b5d585786a9>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  his = model.fit_generator(train,


Epoch 1/50
16/16 [==============================] - ETA: 0s - loss: 26.8646 - accuracy: 0.1895 
Epoch 1: val_accuracy improved from -inf to 0.42383, saving model to best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


16/16 [==============================] - 1047s 67s/step - loss: 26.8646 - accuracy: 0.1895 - val_loss: 12.5558 - val_accuracy: 0.4238
Epoch 2/50
16/16 [==============================] - ETA: 0s - loss: 13.8346 - accuracy: 0.4180 
Epoch 2: val_accuracy improved from 0.42383 to 0.56445, saving model to best_model.h5
16/16 [==============================] - 1089s 70s/step - loss: 13.8346 - accuracy: 0.4180 - val_loss: 10.3321 - val_accuracy: 0.5645
Epoch 3/50
16/16 [==============================] - ETA: 0s - loss: 12.5377 - accuracy: 0.5176 
Epoch 3: val_accuracy improved from 0.56445 to 0.64844, saving model to best_model.h5
16/16 [==============================] - 1032s 67s/step - loss: 12.5377 - accuracy: 0.5176 - val_loss: 8.2323 - val_accuracy: 0.6484
Epoch 4/50
16/16 [==============================] - ETA: 0s - loss: 11.3990 - accuracy: 0.5781 
Epoch 4: val_accuracy improved from 0.64844 to 0.73633, saving model to best_model.h5
16/16 [==============================] - 1035s 67s/st

In [ ]:
from keras.models import load_model
model = load_model("/content/best_model.h5")

In [ ]:
acc = model.evaluate_generator(val)[1]
print(f"The accuracy of your model is = {acc*100} %")

<ipython-input-23-b894b7d2940c>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc = model.evaluate_generator(val)[1]


In [ ]:
def prediction(path):
  img = load_img(path,target_size=(256,256))
  i = img_to_array(img)
  im = preprocess_input(i)
  img = np.expand_dims(im, axis = 0)
  pred = model.predict(img)
  print(pred)

In [ ]:
path = "/content/new-plant-diseases-dataset/test/test/AppleCedarRust1.JPG"
prediction(path)